# TEAM-SASA     
### Initial POC and EDA for Rappler Website

In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

## HELPER CLASS - WEBSCRAPING

In [30]:
DRIVER_PATH = 'C:\Program Files\chromedriver.exe'

class sunstarScraper:

    def __init__(self, search_url):
        self.urls = []
        self.search_url = search_url
        self.driver = webdriver.Chrome(DRIVER_PATH)

    def crawl(self):
        self.driver.get(self.search_url)
        search = self.driver.find_element(By.XPATH, '//*[@id="gsc-i-id1"]')
        search.send_keys('vaccine')
        search.send_keys(Keys.RETURN)
        sleep(5)
        totalpages = self.get_total_pages()
        self.get_pages(totalpages)

    def get_driver(self):
        return self.driver
        
    def get_urls(self):
        return self.urls

    def get_urls_count(self):
        return len(self.urls)

    def get_total_pages(self):
        pages = self.driver.find_elements(By.CSS_SELECTOR, 'div.gsc-cursor-page')
        return len(pages)

    def add_urls(self, page):
        print(f'Start crawling article URLs for page {page}...')
        alinks = self.driver.find_elements(By.CSS_SELECTOR, 'a.gs-title')
        for a in alinks:
            link = a.get_attribute('href')
            if (link != None) and (link not in self.urls):
                self.urls.append(link)
        print('Done.')

    def get_pages(self, totalpages):
        print(f'Total number of pages to crawl: {totalpages}', end='\n\n')
        # Page 1
        self.add_urls(1)
        # Page 2 onwards
        self.driver.maximize_window()
        for p in range(1, totalpages):
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight)', '') 
            pages = self.driver.find_elements(By.CSS_SELECTOR, 'div.gsc-cursor-page')
            pages[p].click()
            sleep(5)
            self.add_urls(p+1)

    def close_popups(self):
        try:
            self.driver.find_element(By.XPATH, '//*[@id="survicate-box"]/div/div/div[3]/div[2]/button').click()
        except:
            pass
        try:
            self.driver.find_element(By.XPATH, '//*[@id="onesignal-slidedown-cancel-button"]').click()
        except:
            pass
        try:
            # Close footer ads
            self.driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[16]//following::div/p/span').click()
        except:
            pass

    def scrape_data(self, url):
        ### PREPARE ARTICLE URL
        self.driver.get(url)
        sleep(15)

        ### PREPARE TITLE
        title = self.driver.find_element(By.CSS_SELECTOR, 'div.titleArticle')
        title = title.text
        print(f'\n[Title]\n{title}')

        ### PREPARE PUBLISHED DATE
        pubdate = self.driver.find_element(By.CSS_SELECTOR, 'div.articleDate')
        pubdate = pubdate.text
        print(f'\n[Date Pulished]\n{pubdate}')

        ### PREPARE AUTHOR NAME
        author = self.driver.find_element(By.CSS_SELECTOR, 'div.articleAuthor')
        author = author.text
        print(f'\n[Author]\n{author}')

        # Try closing popups before clicking 'SEE THE RESULTS' button
        sleep(2)
        self.close_popups()

        ### PREPARE ARTICLE CONTENT
        article = self.driver.find_element(By.CSS_SELECTOR, 'div.articleBody.articleContent.selectionShareable')
        article_content = article.text.replace('ADVERTISEMENT','').replace(' \n','').replace('\n ','').replace('\n\n','').replace('  ','') 
        print(f'\n[Content]\n{article_content}')

        ### PREPARE SENTIMENT RESULTS
        # Scroll to 'SEE THE RESULTS' button then simulate click to get sentiment results
        sleep(3)
        '''
        sentiment_classname = self.driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[16]/div[6]/div[4]/following::div').get_attribute('class')
        self.driver.execute_script(f'document.getElementsByClassName("{sentiment_classname}").scrollIntoView', '')
        
        # Automate click event to get sentiment results
        sentiment_index = 0
        for i in range(100):
            try:
                btn_sentiment = self.driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div[16]/div[6]/div[4]/div[{i}]/div/div/span')
                if btn_sentiment.text.upper() == 'SEE THE RESULTS':
                    sleep(2)
                    self.close_popups()
                    action = ActionChains(self.driver)
                    action.move_to_element(btn_sentiment).click().perform()
                    # btn_sentiment.click()
                    sentiment_index = i
                    break
            except:
                pass

        sleep(3)
        # Clean some sentiment texts
        sentiment_results = self.driver.find_element(By.XPATH, f'//*[@id="__next"]/div/div[16]/div[6]/div[4]/div[{sentiment_index}]/div')
        sentiment_results = sentiment_results.text.replace('HOW OTHERS ARE FEELING', '').replace(' \n','').replace('\n ','').replace(' ARE ','-')
        # Convert each line of sentiment text to a list item
        sentiment_results = list(filter(None, sentiment_results.split('\n')))
        # Make a key-value pair of sentiment text and store it as dictionary object
        sentiment_results = {s.split('-')[1] : float(s.split('-')[0].split('%')[0])*.01 for s in sentiment_results}
        # Add a 0% value for sentiment key that does not have results in the current article and store as 'other sentiments'
        sentiment_keys = ['INSPIRED', 'HAPPY', 'AMUSED', 'NEUTRAL', 'ANNOYED', 'SAD', 'ANGRY', 'AFRAID']
        sentiment_others = {k : 0. for k in sentiment_keys if k not in sentiment_results}
        # Combine 'other sentiment dictionary' to the 'sentiment result dictionary' 
        sentiment_results = {**sentiment_results, **sentiment_others}
        print(f'\n[Sentiment]')
        print(sentiment_results)
        '''

        ### GET SENTIMENT RESULTS FROM ARTICLE
        return {'urls':url, 'titles':title, 'authors':author, 'published_dates':pubdate, 'articles_content':article_content}
    

# RUN WEB SCRAPING

In [31]:
KEYWORD = 'vaccine'
SEARCH_URL = f'https://www.sunstar.com.ph/search'

scraper = sunstarScraper(SEARCH_URL)
scraper.crawl()

dataset = []
print(f'\nTotal number of articles: {scraper.get_urls_count()}')
for i,url in enumerate(scraper.get_urls()):
    try:
        print(f'\n\n\n[{i+1}]{" ":>5}SCRAPING DATA  ~>  {url}')
        data = scraper.scrape_data(url)
        dataset.append(data)
    except Exception as e:
        print(f'SCRAPING ERROR: {e}')
        pass

scraper.get_driver().quit()

Total number of pages to crawl: 10

Start crawling article URLs for page 1...
Done.
Start crawling article URLs for page 2...
Done.
Start crawling article URLs for page 3...
Done.
Start crawling article URLs for page 4...
Done.
Start crawling article URLs for page 5...
Done.
Start crawling article URLs for page 6...
Done.
Start crawling article URLs for page 7...
Done.
Start crawling article URLs for page 8...
Done.
Start crawling article URLs for page 9...
Done.
Start crawling article URLs for page 10...
Done.

Total number of articles: 100



[1]     SCRAPING DATA  ~>  https://www.sunstar.com.ph/article/1887620/Cebu/Local-News/768-of-3000-VSMMC-frontliners-to-try-Chinas-Sinovac-vaccine

[Title]
768 of 3,000 VSMMC frontliners to try China's Sinovac vaccine

[Date Pulished]
March 02, 2021

[Author]


[Content]
TO INCREASE public confidence in the Sinovac vaccine donated by the Chinese government, retired general Melquiades Feliciano and Dr. Gerardo Aquino Jr. will be the first to be in




[3]     SCRAPING DATA  ~>  https://www.sunstar.com.ph/article/1887844/Cebu/Local-News/First-vaccinations

[Title]
More VSMMC healthcare workers say yes to Sinovac

[Date Pulished]
March 04, 2021

[Author]
WENILYN SABALO

[Content]
THOUGH hesitant at first, the number of healthcare workers (HCWs) at the Vicente Sotto Memorial Medical Center (VSMMC) in Cebu City who consented to be inoculated with Sinovac’s CoronaVac Covid-19 vaccine has increased as of Wednesday, March 3, 2021.VSMMC chief Dr. Gerardo Aquino Jr. said 1,246 HCWs of the hospital have already agreed to be vaccinated.
Aquino, 56, was the first to get the Sinovac vaccine during a ceremony for the first vaccine rollout in the Visayas and Mindanao held inside the new Center for Behavioral Sciences building of the VSMMC at 2:30 p.m. on Thursday, March 4.After Aquino, Dr. Juanito Tiu of St. Anthony Mother and Child Hospital, Dr. Pythagoras Zerna of Eversley Childs Sanitarium and General Hospital, Ret. Maj. Gen. Melquiades Feli




[4]     SCRAPING DATA  ~>  https://www.sunstar.com.ph/article/1879955/Tacloban/Feature/Filipino-Dominican-priest-scientist-works-on-affordable-yeast-based-Covid-19-vaccine

[Title]
Filipino Dominican priest-scientist works on affordable yeast-based Covid-19 vaccine

[Date Pulished]
December 13, 2020

[Author]
RONALD O. REYES

[Content]
AS SCIENTISTS and pharmaceutical companies in rich countries around the world are racing for the development and application of vaccine against the killer coronavirus disease (Covid-19), a Filipino Dominican priest-microbiologist based in the United States has also decided to make a vaccine that will be affordable and accessible to other middle to low income countries, including the Philippines.“I thought about developing this vaccine when I realized that many of the available vaccines being manufactured today have already been promised to resource-rich countries,” said Father Nicanor Austriaco, chief researcher at Austriaco Laboratory.
Austriaco, who




[6]     SCRAPING DATA  ~>  https://www.sunstar.com.ph/article/1882627/Manila/Local-News/Quezon-Makati-lead-LGUs-with-AstraZeneca-vaccine-deals

[Title]
Quezon, Makati lead LGUs with AstraZeneca vaccine deals

[Date Pulished]
January 13, 2021

[Author]


[Content]
THE cities of Quezon and Makati in Metro Manila are leading the list of local government units (LGUs) that have signed advance purchase agreements with multinational pharmaceutical company AstraZeneca for the procurement of its candidate vaccine against coronavirus disease 2019 (Covid-19).Quezon City announced on Monday, January 11, 2021 that it was increasing its purchase order to 1.1 million doses of the AZD1222 vaccine from the initial 550,000 doses.
Makati City Mayor Abigail Binay, on the other hand, announced on Tuesday, January 12, that the City Government is procuring 1.0 million doses.They are among the more than 20 LGUs that have signed agreements with AstraZeneca, National Task Force Against Covid-19 and Departmen




[8]     SCRAPING DATA  ~>  https://www.sunstar.com.ph/article/1884638/Manila/Local-News/Special-Report-Government-prepares-for-vaccine-rollout

[Title]
Special Report: Government prepares for vaccine rollout

[Date Pulished]
February 02, 2021

[Author]
THIRD ANNE PERALTA-MALONZO

[Content]
(This is the third part of a special report by SunStar Philippines on efforts to defeat the Covid-19 pandemic through vaccination.)NEARLY 24.7 million Filipinos, or about 23 percent of the population, will receive the free vaccine against coronavirus disease 2019 (Covid-19) first.
Among them are the frontline health workers, senior citizens, indigent population and the uniformed personnel, Department of Health (DOH) Undersecretary Myrna Cabotaje said.The priority list, or Group A, includes:
1,762,994 frontline health workers in both public and private hospitals, and barangay health workers;
3,789,874 indigent senior citizens;
5,678,544 remaining senior citizens;
12,911,193 indigent population; and

...


[Content]
THE common response to a recent survey on whether they would take the vaccine against Covid-19 was that of wait and see–wait for what will happen before we decide, or let others be first then we will see.Well, the list of people wanting to go first to help boost confidence in the vaccination program against the coronavirus disease (Covid-19) is growing longer. And they’re willing to do it in public. Perhaps à la influencer on social media for better impact and wider reach.
The government’s vaccination program is working on a priority list based on World Health Organization (WHO) guidelines. At the top of the list are medical frontliners who have to be strong enough to care for the sick, followed by the elderly, indigent and law enforcers.To try to convince majority of Filipinos to have themselves vaccinated, there are local government officials and Church leaders and members who are offering themselves to be among the first to get inoculated in order to show others that the 




[88]     SCRAPING DATA  ~>  https://www.sunstar.com.ph/article/1883884/Cebu/Local-News/DOH-Central-Visayas-works-on-having-Covid-19-vaccine-freezers

[Title]
DOH-Central Visayas works on having Covid-19 vaccine freezers

[Date Pulished]
January 25, 2021

[Author]


[Content]
THE Department of Health Central Visayas (DOH 7) is working on acquiring appropriate bio-freezers for the incoming Covid-19 vaccines.Dr. Mary Jean Loreche, DOH 7 spokesperson, said there is no specific number of doses or a specific vaccine brand to be used in Cebu yet, but DOH 7’s preparations are ongoing. She assured the public that they will hasten this.
Loreche said there is a need to assess the region’s storage capabilities and they are doing inventory on the number of freezers that have a 220-degree capacity and those ultra-low freezers that have -70 degrees.“We will try to act fast on this so that when the vaccines are ready, we have a place for them,” she said.She said there are institutions in the region




[90]     SCRAPING DATA  ~>  https://www.sunstar.com.ph/article/1881368/Manila/Local-News/Importers-distributors-of-Sinopharm-vaccine-face-sanctions

[Title]
Importers, distributors of Sinopharm vaccine face sanctions

[Date Pulished]
December 29, 2020

[Author]


[Content]
HEALTH officials on Tuesday, December 29, 2020, said the people who imported and distributed the unregistered Sinopharm vaccine against coronavirus disease 2019 (Covid-19) will face sanctions.Department of Health (DOH) Undersecretary Maria Rosario Vergeire said the sale, distribution or supply of a product that is not registered with the Food and Drug Administration (FDA) violate Republic Act 9711.
"Those who brought in this vaccine, those who have supplied or distributed, and even the manufacturer may be sanctioned if they are used by our population without the proper processing by our regulatory agency, which is FDA,” Vergeire said in a virtual press briefing on Tuesday.Asked whether President Rodrigo Duterte or




[94]     SCRAPING DATA  ~>  https://www.sunstar.com.ph/article/1885368/Manila/Local-News/Poe-Vaccine-passports-should-be-free-for-all-Filipinos

[Title]
Poe: Vaccine passports should be free for all Filipinos

[Date Pulished]
February 09, 2021

[Author]


[Content]
SENATOR Grace Poe has reiterated that the proposed vaccine passports should be free for all Filipinos."Talagang dapat libre ito. Sa batas namin libre ito," said Poe.
Section 10 of Poe's proposed "Vaccine Passport Act" (S. No. 1994) states no fees shall be collected for the issuance, amendment, or replacement of a vaccine passport. The bill remains pending in the Senate Committee on Health and Demography."Cost should not be a consideration in restoring normalcy and providing peace of mind to Filipinos," said Poe in the bill's explanatory note.Poe also noted that the Department of Health (DOH) fully supported the rollout of a vaccine passport even without the enactment of a law from Congress.
"Nung kami'y nagkaroon ng pagdi




[97]     SCRAPING DATA  ~>  https://www.sunstar.com.ph/article/1884475/Pampanga/Local-News/Guagua-mayor-says-information-drive-key-to-Covid-19-vaccine-acceptability

[Title]
Guagua mayor says information drive key to Covid 19 vaccine acceptability

[Date Pulished]
January 31, 2021

[Author]
IAN OCAMPO FLORA

[Content]
DEBUNKING myths and biases towards vaccines is a key approach in ensuring that any vaccination plan is acceptable to the public, according to Guagua town Mayor Dante Torres, whose team had been conceptualizing a virus vaccination plan for residents of his town.Torres had initially allocated some P5 million for the purchase of coronavirus vaccines but was quick to note that while the country is awaiting available supply, towns should now set their attention towards preparing a vaccination road map that includes a massive information campaign on the importance of vaccines in general.
“There is no use if we have a vaccine when a considerable number of the vulnerable and p

# EDA

In [32]:
df = pd.DataFrame(dataset)
df

,urls,titles,authors,published_dates,articles_content
0,https://www.sunstar.com.ph/article/1887620/Ceb...,"768 of 3,000 VSMMC frontliners to try China's ...",,"March 02, 2021",TO INCREASE public confidence in the Sinovac v...
1,https://www.sunstar.com.ph/article/1883079/Dav...,#AllYouNeedToKnow: AstraZeneca's Covid-19 vaccine,REUEL JOHN F. LUMAWAG,"January 17, 2021",DAVAO City Mayor Sara Z. Duterte-Carpio announ...
2,https://www.sunstar.com.ph/article/1887844/Ceb...,More VSMMC healthcare workers say yes to Sinovac,WENILYN SABALO,"March 04, 2021","THOUGH hesitant at first, the number of health..."
3,https://www.sunstar.com.ph/article/1879955/Tac...,Filipino Dominican priest-scientist works on a...,RONALD O. REYES,"December 13, 2020",AS SCIENTISTS and pharmaceutical companies in ...
4,https://www.sunstar.com.ph/article/1882990/Ceb...,Cebu City prepares to procure vaccine for coro...,,"January 16, 2021",WITH the Food and Drugs Administration (FDA) g...
...,...,...,...,...,...
95,https://www.sunstar.com.ph/article/1866696/Net...,Scientists uneasy as Russia approves 1st coron...,,"August 12, 2020","MOSCOW -- Russia on Tuesday, August 11, became..."
96,https://www.sunstar.com.ph/article/1884475/Pam...,Guagua mayor says information drive key to Cov...,IAN OCAMPO FLORA,"January 31, 2021",DEBUNKING myths and biases towards vaccines is...
97,https://www.sunstar.com.ph/article/1882632/Net...,India starts shipping Covid-19 vaccine to cities,,"January 13, 2021",NEW DELHI -- India has started shipping Covid-...
98,https://www.sunstar.com.ph/article/1879248/Man...,Duque willing to get Covid-19 vaccine first to...,THIRD ANNE PERALTA-MALONZO,"December 07, 2020",DEPARTMENT of Health (DOH) Secretary Francisco...


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   urls              100 non-null    object
 1   titles            100 non-null    object
 2   authors           100 non-null    object
 3   published_dates   100 non-null    object
 4   articles_content  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [34]:
df.describe().T

,count,unique,top,freq
urls,100,100,https://www.sunstar.com.ph/article/1885574/Ceb...,1
titles,100,100,Cathay Pacific Cargo develops solution for vac...,1
authors,100,20,,64
published_dates,100,62,"January 15, 2021",4
articles_content,100,100,RUSSIA has claimed that it has already produce...,1


In [35]:
df.isna().sum()

urls                0
titles              0
authors             0
published_dates     0
articles_content    0
dtype: int64

In [36]:
### Save a CSV dataset
df.to_csv(f'sunstar_{KEYWORD}.csv', index=False)

# WORDCLOUD

In [37]:
from wordcloud import WordCloud
from PIL import Image
import numpy as np

ModuleNotFoundError: No module named 'wordcloud.query_integral_image'

In [ ]:
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % np.random.randint(60, 100)

mask = np.array(Image.open('bg.png'))

words = ' '.join(df['articles_content'].tolist())
wordcloud = WordCloud(background_color='black', max_words=2000, width=1024, height=720, mask=mask)
wordcloud.generate(words)

plt.figure(figsize=(20, 8), dpi=300)
plt.title('Rappler on COVID Vaccine in PH (Word Cloud)')
plt.imshow(wordcloud.recolor(color_func=grey_color_func, random_state=42), interpolation='bilinear')
plt.axis('off')
plt.show()